In [172]:
import pandas as pd
import glob
import requests
from rake_nltk import Metric, Rake
from bs4 import BeautifulSoup
import string
from transformers import pipeline
from nltk.corpus import wordnet
import nltk
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [96]:
import time

In [3]:
taxList = glob.glob("./CSV Files/*taxonomy.csv")
assList = glob.glob("./CSV Files/*assessments.csv")

In [103]:
dftList = []
for file in taxList:
    dftList.append(pd.read_csv(file))
dfaList = []
for file in assList:
    dfaList.append(pd.read_csv(file))
    
dfTotal = pd.concat(dftList)
dfAssess = pd.concat(dfaList)
dfAssess.reset_index(inplace=True, drop=True)

In [104]:
dfTotalClean = dfTotal[["internalTaxonId", "speciesName", "className", "familyName", "scientificName"]]
dfTotalClean.head()
dfAssessClean = dfAssess[["assessmentId", "internalTaxonId", "redlistCategory", "yearPublished", "populationTrend", "systems", "realm", "yearLastSeen", "scopes"]]
dfAssessClean.head()

,assessmentId,internalTaxonId,redlistCategory,yearPublished,populationTrend,systems,realm,yearLastSeen,scopes
0,495907,10041,Critically Endangered,2021,Decreasing,Terrestrial|Freshwater (=Inland waters),Indomalayan,NaN,Global
1,499158,10825,Critically Endangered,2021,Decreasing,Terrestrial,Australasian,NaN,Global
2,499618,10950,Critically Endangered,2021,Decreasing,Terrestrial|Freshwater (=Inland waters),Indomalayan,NaN,Global
3,508210,12696,Critically Endangered,2019,Decreasing,Terrestrial,Afrotropical,NaN,Global
4,516369,135889,Critically Endangered,2020,Decreasing,Terrestrial|Freshwater (=Inland waters),Neotropical,NaN,Global


In [123]:
dfAssessClean.to_csv("assessments.csv")

url = "https://api.gbif.org/v1/species?language=EN&nameType=SCIENTIFIC&name=Heosemys annandalii"
results = requests.get(url).text
uName = results[results.find("\"vernacularName") : results.find(",\"authorship")]
cName = uName[uName.find(":")+1:-1].replace("\"", "")
print(cName)

In [ ]:
for index, row in dfTotalClean.iterrows():
    url = "https://api.gbif.org/v1/species?language=EN&nameType=SCIENTIFIC&name=" + row['scientificName']
    results = requests.get(url).text
    uName = results[results.find("\"vernacularName") : results.find(",\"authorship")]
    cName = uName[uName.find(":")+1:-1].replace("\"", "")
    dfTotalClean.loc[index, "commonName"] = cName
    time.sleep(.01)
dfTotalClean.head()

In [102]:
dfTotalClean.reset_index(inplace=True, drop=True)
dfTotalClean.to_csv("species.csv")

In [105]:
allThreats = dfAssess['threats'].tolist()
threats = []
for x in allThreats:
    threats.append(BeautifulSoup(str(x), 'html.parser').get_text().replace("(", "").replace(")", "")) 

print(len(threats))

9129


In [106]:
s = Rake(punctuations="““’‘‘––().,;:></?'!@#$%^&*-_=+")
s.extract_keywords_from_text("".join(threats))
swordD = s.get_word_frequency_distribution()
sworddf = pd.DataFrame.from_dict(swordD, orient='index')
sworddf = sworddf.reset_index().rename(columns={'index':'word', 0:'count'})
rworddf = sworddf[sworddf['count'] >= 10].sort_values(by=['count'], ascending=False).reset_index(drop=True)

In [107]:
results = classifier(rworddf.word.tolist())

In [108]:
rdf = pd.DataFrame(results)
rdf = pd.concat([rworddf, rdf], axis=1)
nwDF = rdf[rdf['label'] == "NEGATIVE"].reset_index(drop=True)
nwDF.to_csv("test4.csv")

In [ ]:
words = pd.read_csv("test4.csv")

keywords = words["word"].to_list()
keywords.remove("p")
keywords.remove("f")
keywords.remove("threat")
keywords.remove("threats")
keywords.remove("threatened")
keywords.remove("threaten")
keywords.remove("0")
keywords.remove("comm")
keywords.append("Human")
keywords.append("human")
keywords.append("Deforestation")
keywords.append("deforestation")
keywords.append("Habitat loss")
keywords.append("habitat loss")
keywords.append("logging")
keywords.append("forest loss")
keywords.append("agriculture")
keywords.append("mining")
for x in range(2025):
    if str(x) in keywords:
        keywords.remove(str(x))
for x in keywords:
    if len(str(x)) < 3:
        keywords.remove(x)

print(keywords)

In [121]:
dfThreats = dfAssess[['internalTaxonId','threats']]

for index, row in dfThreats.iterrows():
    sL = ""
    rowString = BeautifulSoup(str(row['threats']), 'html.parser').get_text().replace("(", "").replace(")", "").lower()
    for word in keywords:
        if word in rowString:
            sL += word + ", "
    dfThreats.at[index, "Threat Keywords"] = sL
dfThreats.head()

C:\Python39\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,internalTaxonId,threats,Threat Keywords
0,10041,<p>The main threats to this species are harves...,"loss, however, illegal, rat, viet, late, trade..."
1,10825,Current threats include severe and rampant hab...,"logging, red, destruction, road, reduced, seve..."
2,10950,<p>Widespread local subsistence consumption of...,"small, illegal, fragmented, dam, rat, erosion,..."
3,12696,The single most serious human threat to <em>Ma...,"small, degradation, illegal, red, predation, d..."
4,135889,"Local inhabitants' comments to I. De la Riva, ...","declines, although, decline, fungus, severe, c..."


In [122]:
dfThreats.to_csv("threatTest2.csv")

r = Rake(max_length=2, punctuations="““’‘‘––().,;:></?'!@#$%^&*-_=+")
r.extract_keywords_from_sentences(threats)

wordD = r.get_word_frequency_distribution()
worddf = pd.DataFrame.from_dict(wordD, orient='index')
worddf.to_csv("test2.csv")

g = Rake(include_repeated_phrases=False, punctuations="““’‘‘––().,;:></?'!@#$%^&*-_=+")
g.extract_keywords_from_text("".join(threats))
results2 = classifier(g.get_ranked_phrases())

r2df = pd.DataFrame(results)
r2df = pd.concat([r2df, pd.Series(g.get_ranked_phrases())], axis=1)
r2df = r2df[r2df['label'] == "NEGATIVE"].reset_index(drop=True)
r2df.to_csv("test5.csv")

In [112]:
allConserve = dfAssess['conservationActions'].tolist()
conserve = []
for x in allConserve:
    conserve.append(BeautifulSoup(str(x), 'html.parser').get_text().replace("(", "").replace(")", "")) 

print(len(conserve))

9129


In [113]:
c = Rake(punctuations="““’‘‘––().,;:></?'!@#$%^&*-_=+")
c.extract_keywords_from_text("".join(conserve))
cwordD = c.get_word_frequency_distribution()
cworddf = pd.DataFrame.from_dict(cwordD, orient='index')
cworddf = cworddf.reset_index().rename(columns={'index':'word', 0:'count'})
cworddf = cworddf[cworddf['count'] >= 10].sort_values(by=['count'], ascending=False).reset_index(drop=True)

In [114]:
results3 = classifier(cworddf.word.tolist())

In [59]:
cdf = pd.DataFrame(results3)
cdf = pd.concat([cworddf, cdf], axis=1)
nwcDF = cdf[cdf['label'] == "POSITIVE"].reset_index(drop=True)
nwcDF.to_csv("ctest1.csv")

In [ ]:
cwords = pd.read_csv("ctest1.csv")

ckeywords = cwords["word"].to_list()
ckeywords.remove("species")
ckeywords.remove("et")
ckeywords.remove("al")
ckeywords.remove("litt")
ckeywords.remove("pers")
ckeywords.remove("also")
ckeywords.remove("iii")
ckeywords.remove("://")
ckeywords.remove("nam")
ckeywords.remove("conservation")
ckeywords.remove("actions")
ckeywords.remove("appendix")
ckeywords.remove("cites")
ckeywords.remove("area")
ckeywords.remove("areas")
ckeywords.append("no conservation measures")
for x in range(2025):
    if str(x) in ckeywords:
        ckeywords.remove(str(x))
for x in ckeywords:
    if len(str(x)) < 3:
        ckeywords.remove(x)
print(ckeywords)

In [ ]:
dfConserve = dfAssess[['internalTaxonId','conservationActions']]

for index, row in dfConserve.iterrows():
    sL = ""
    rowString = BeautifulSoup(str(row['conservationActions']), 'html.parser').get_text().replace("(", "").replace(")", "").lower()
    for word in ckeywords:
        if str(word) in rowString:
            sL += str(word) + ", "
    dfConserve.at[index, "Conserve Keywords"] = sL
dfConserve.head()

In [117]:
dfConserve.to_csv("conserveTest.csv")

In [ ]:
dfHabitat = dfAssess[['internalTaxonId', 'habitat', 'systems']]
cleanedRows = []
for index, row in dfHabitat.iterrows():
    rowString = BeautifulSoup(str(row['habitat']), 'html.parser').get_text().replace("(", "").replace(")", "").lower()
    dfHabitat.at[index, "habitat_type"] = rowString[:rowString.find(".")+1]
dfHabitat.head()

In [119]:
dfHabitat.to_csv("habitatTest.csv")

In [ ]:
summarizer = pipeline("summarization")

In [145]:
summ = []
for x in threats[:5]:
    wrdlen = len(x.split(" "))
    summ += summarizer(x, max_length=wrdlen)

In [ ]:
summL = []
for x in summ:
    summary = list(x.values())
    summL.append(summary[0])
print(summL)

In [173]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\RaininSwords-
[nltk_data]     Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [397]:
humans = wordnet.synsets("human")
natural = wordnet.synsets("natural")
genetics = wordnet.synsets("genetics")
disease = wordnet.synsets("disease")
tcategoryList = [humans, natural, genetics, disease]
str(genetics[0].lemmas()[0].name())

'genetics'

In [462]:
first = wordnet.synsets("pollution")
second = wordnet.synsets("degradation")
print(first, second)

avg = 0
count = 0
for x in first:
    for y in second:
        if y.wup_similarity(x) > .2:
            avg += y.wup_similarity(x)
            count += 1
print(avg/count)

[Synset('pollution.n.01'), Synset('befoulment.n.01'), Synset('contamination.n.03')] [Synset('degradation.n.01'), Synset('abasement.n.01')]
0.4484864121087031


In [465]:
cleanThreats = pd.read_csv("allThreats.csv")
cleanThreats.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
cleanThreats.set_index('Index', drop=True, inplace=True)
cleanThreats = cleanThreats[["internalTaxonId", "Threat Keywords"]]
for index, row in cleanThreats.iterrows():
    keyList = str(row['Threat Keywords']).split(",")
    for s in tcategoryList:
        boolean = False
        cString = ""
        if s == humans:
            cString = "Humans"
        if s == natural:
            cString = "Natural"
        if s == genetics:
            cString = "Genes"
        if s == disease:
            cString = "Diseases"

        for k in keyList:
            avg = 0
            count = 0
            temp = wordnet.synsets(k.strip())
            for x in temp:
                for y in s:
                    sim = y.wup_similarity(x)
                    if sim > .35:
                        avg += sim
                        count += 1
                        
            if count > 0:
                avg = avg/count
                if avg > .45:
                    boolean = True
        
        cleanThreats.at[index, cString] = boolean

cleanThreats.head(20)

,internalTaxonId,Threat Keywords,Humans,Natural,Genes,Diseases
Index,,,,,,
0,10041,"loss, however, illegal, rat, viet, late, trade...",True,True,True,True
1,10825,"logging, red, destruction, road, reduced, seve...",True,True,True,True
2,10950,"small, illegal, fragmented, dam, rat, erosion,...",True,True,True,True
3,12696,"small, degradation, illegal, red, predation, d...",True,True,False,True
4,135889,"declines, although, decline, fungus, severe, c...",True,True,True,True
5,136773,"loss, hunting, deforestation, although, red, l...",True,True,False,True
6,14765,"mining, illegal, red, low, exploitation, trade...",True,True,False,True
7,15026,"loss, degradation, low, pressure, exploitation...",True,True,False,True
8,15509,"despite, traded, exported, pit, anti, habit, d...",True,True,False,True


In [466]:
cleanThreats.to_csv("allThreatsCorrect.csv")

In [467]:
protection = wordnet.synsets("protection")
speciesManagment = wordnet.synsets("breeding")
education = wordnet.synsets("education")
tradeCtrl = wordnet.synsets("trade")
research = wordnet.synsets("research")
ccategoryList = [protection, speciesManagment, education, tradeCtrl, research]

In [ ]:
cleanActions = pd.read_csv("allConservationActions.csv")
cleanActions.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
cleanActions.set_index('Index', drop=True, inplace=True)
cleanActions = cleanActions[["internalTaxonId", "Conserve Keywords"]]
for index, row in cleanActions.iterrows():
    keyList = str(row['Conserve Keywords']).split(",")
    for s in ccategoryList:
        boolean = False
        cString = ""
        if s == protection:
            cString = "land_water_protection"
        if s == speciesManagment:
            cString = "species_management"
        if s == education:
            cString = "education"
        if s == tradeCtrl:
            cString = "trade_control"
        if s == research:
            cString = "research_monitoring"

        for k in keyList:
            avg = 0
            count = 0
            temp = wordnet.synsets(k.strip())
            for x in temp:
                for y in s:
                    sim = y.wup_similarity(x)
                    if sim > .30:
                        avg += sim
                        count += 1
                        
            if count > 0:
                avg = avg/count
                if avg > .50:
                    boolean = True
        
        cleanActions.at[index, cString] = boolean

cleanActions.head(20)

In [472]:
cleanActions.to_csv("allConservationActionsCorrect.csv")

In [482]:
cleanclean = cleanActions.replace([True, False], [1, 0])
cleanclean = pd.concat([cleanclean, dfAssess['assessmentId']], axis=1)
cleanclean.reset_index(inplace=True)
cleanclean.to_csv("conservationActionsCorrect.csv")

In [480]:
specConserve = cleanclean[['internalTaxonId']]
specConserve.reset_index(inplace=True)
specConserve.to_csv("species_conservation.csv")

In [484]:
specThreats = cleanThreats[['internalTaxonId']]
specThreats.reset_index(inplace=True)
specThreats.to_csv("species_threats.csv")

In [487]:
specHabitats = dfHabitat[['internalTaxonId']]
specHabitats.reset_index(inplace=True)
specHabitats.to_csv("species_habitat.csv")

In [490]:
alist = dfAssessClean['redlistCategory'].to_list()
uaList = []
for x in alist:
    if x not in uaList:
        uaList.append(x)
print(uaList)

['Critically Endangered', 'Endangered', 'Near Threatened', 'Lower Risk/near threatened', 'Vulnerable']


In [498]:
dfAssessC = dfAssessClean.replace(['Critically Endangered', 'Endangered', 'Near Threatened', 'Lower Risk/near threatened', 'Vulnerable'], [0, 1, 2, 3, 4])
dfAssessC.to_csv("species_assessment.csv")

In [497]:
statusD = {'status_id': [0, 1, 2, 3, 4], 'status_name' : ['Critically Endangered', 'Endangered', 'Near Threatened', 'Lower Risk/near threatened', 'Vulnerable']}
statusdf = pd.DataFrame.from_dict(statusD)
statusdf.to_csv("status.csv")

In [500]:
cleancleanT = cleanThreats.replace([True, False], [1, 0])
cleancleanT.to_csv("threats.csv")